In [ ]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns 
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()

In [ ]:
train_df=pd.read_csv('../input/train.csv')
test_df=pd.read_csv('../input/test.csv')

In [ ]:
pd.set_option('display.max_columns', 300)
train_df.head()

In [ ]:
test_df.head()

EDA
======

In [ ]:
sns.countplot(train_df['Target']);
plt.xticks(rotation=45);
plt.title('Income levels')
plt.show()

In [ ]:
print('There are {0}% people in the group of extreme poverty.'.format(np.round(train_df.Target.value_counts(normalize=True).values[3], 4) * 100))
print('There are {0}% people in the group of non vulnerable households.'.format(np.round(train_df.Target.value_counts(normalize=True).values[0], 4) * 100))

In [ ]:
sns.set(font_scale=1, style="darkgrid")
c =  sns.color_palette('spring_d')[4]
sns_jointplot = sns.jointplot('age', 'meaneduc', data=train_df, kind='kde', color=c, size=6)

In [ ]:
train_df['tipovivi3'].value_counts()

In [ ]:
# hourse quality
df_q = train_df[['Target', 'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3']]
df_q.loc[df_q['epared1'] == 1, 'wall'] = 'Bad'
df_q.loc[df_q['epared2'] == 1, 'wall'] = 'Regular'
df_q.loc[df_q['epared3'] == 1, 'wall'] = 'Good'

df_q.loc[df_q['etecho1'] == 1, 'roof'] = 'Bad'
df_q.loc[df_q['etecho2'] == 1, 'roof'] = 'Regular'
df_q.loc[df_q['etecho3'] == 1, 'roof'] = 'Good'

df_q.loc[df_q['eviv1'] == 1, 'floor'] = 'Bad'
df_q.loc[df_q['eviv2'] == 1, 'floor'] = 'Regular'
df_q.loc[df_q['eviv3'] == 1, 'floor'] = 'Good'

df_q = df_q[['Target', 'wall', 'roof', 'floor']]

In [ ]:
print("Roof quality")
df_q.loc[df_q['Target'] == 1, 'Target'] = 'Extreme'
df_q.loc[df_q['Target'] == 2,'Target'] = 'Moderate'
df_q.loc[df_q['Target'] == 3,'Target'] = 'Vulnerable'
df_q.loc[df_q['Target'] == 4,'Target'] = 'Non-Vulnerable'
ax = sns.catplot(x = 'roof', col = 'Target', data = df_q, kind="count", col_order=['Extreme', 'Moderate', 'Vulnerable', 'Non-Vulnerable']).set_titles("{col_name}")
ax.fig.set_size_inches(15,4)
ax.set(ylabel = '')
plt.show()

In [ ]:
print("Wall quality")
ax = sns.catplot(x = 'wall', col = 'Target', data = df_q, kind="count" ,col_order=['Extreme', 'Moderate', 'Vulnerable', 'Non-Vulnerable'], order = ['Bad', 'Regular', 'Good']).set_titles("{col_name}")
ax.fig.set_size_inches(15,4)
ax.set(ylabel = '')
plt.show()

In [ ]:
print("Floor quality")
ax = sns.catplot(x = 'floor', col = 'Target', data = df_q, kind="count", col_order=['Extreme', 'Moderate', 'Vulnerable', 'Non-Vulnerable']).set_titles("{col_name}")
ax.fig.set_size_inches(15,4)
ax.set(ylabel = '')
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(x='Target', y = 'r4h3',ax = ax, data = train_df, hue = 'Target' )
ax.set_title('Number of men in households', size = 25)

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(x='Target', y = 'r4m3',ax = ax, data = train_df, hue = 'Target' )
ax.set_title('Number of women in households', size = 25)

In [ ]:
ninos = train_df.groupby(by = 'Target')['hogar_nin', 'Target'].sum()
ninos = pd.DataFrame(ninos)
ninos['mean_children'] = (ninos['hogar_nin']/ninos['Target'])
ninos['Target1'] = ninos.index.map({4: "NonVulnerable", 3: "Moderate Poverty", 2: "Vulnerable", 1: "Extereme Poverty"})
sns.set(style = 'whitegrid', font_scale=1.4)
fig = plt.subplots(figsize=(10, 6))
ax = sns.barplot(x = 'Target1', y = 'mean_children', data = ninos, palette='Pastel1', ci = None).set_title('Mean number on children in different households')

Feature Engineering
==========

In [ ]:
# change categorical features
categorical_columns = [col for col in train_df.columns if train_df[col].dtype == 'object']
categorical_columns

In [ ]:
cols=['dependency', 'edjefe', 'edjefa']

In [ ]:
train_df[cols] = train_df[cols].replace({'no': 0, 'yes':1}).astype(float)
test_df[cols] = test_df[cols].replace({'no': 0, 'yes':1}).astype(float)
train_df['dependency'] = train_df['dependency'].replace({np.inf: 0})
test_df['dependency'] = test_df['dependency'].replace({np.inf: 0})

In [ ]:
test_df.head(20)

In [ ]:
# check missing values
total = train_df.isnull().sum().sort_values(ascending = False)
percent = (train_df.isnull().sum()/train_df.isnull().count()*100).sort_values(ascending = False)
missing_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_train_data.head(15)

In [ ]:
train_df.corr().sort_values('Target', ascending=False)['Target']

In [ ]:
data = [
    go.Heatmap(
        z= train_df.corr().values,
        x=train_df.columns.values,
        y=train_df.columns.values,
        colorscale='Viridis',
        reversescale = False,
        #text = True ,
        opacity = 1.0 )
]

layout = go.Layout(
    title='Pearson Correlation of features',
    xaxis = dict(ticks='', nticks=36),
    yaxis = dict(ticks='' ),
    width = 900, height = 700,
margin=dict(
    l=240,
),)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')

In [ ]:
#Fill na
def repalce_v18q1(x):
    if x['v18q'] == 0:
        return x['v18q']
    else:
        return x['v18q1']

train_df['v18q1'] = train_df.apply(lambda x : repalce_v18q1(x),axis=1)
test_df['v18q1'] = test_df.apply(lambda x : repalce_v18q1(x),axis=1)

train_df['v2a1'] = train_df['v2a1'].fillna(value=train_df['tipovivi3'])
test_df['v2a1'] = test_df['v2a1'].fillna(value=test_df['tipovivi3'])

In [ ]:
#add other feats
train_df['roof_waste_material'] = np.nan
test_df['roof_waste_material'] = np.nan
train_df['electricity_other'] = np.nan
test_df['electricity_other'] = np.nan

def fill_roof_exception(x):
    if (x['techozinc'] == 0) and (x['techoentrepiso'] == 0) and (x['techocane'] == 0) and (x['techootro'] == 0):
        return 1
    else:
        return 0
    
def fill_no_electricity(x):
    if (x['public'] == 0) and (x['planpri'] == 0) and (x['noelec'] == 0) and (x['coopele'] == 0):
        return 1
    else:
        return 0

train_df['roof_waste_material'] = train_df.apply(lambda x : fill_roof_exception(x),axis=1)
test_df['roof_waste_material'] = test_df.apply(lambda x : fill_roof_exception(x),axis=1)
train_df['electricity_other'] = train_df.apply(lambda x : fill_no_electricity(x),axis=1)
test_df['electricity_other'] = test_df.apply(lambda x : fill_no_electricity(x),axis=1)

In [ ]:
def owner_is_adult(x):
    if x['age'] <= 18:
        return 0
    else:
        return 1

train_df['head_below18'] = train_df.apply(lambda x : owner_is_adult(x),axis=1)
test_df['head_below18'] = test_df.apply(lambda x : owner_is_adult(x),axis=1)

In [ ]:
train_df['adult'] = train_df['hogar_adul'] - train_df['hogar_mayor']
train_df['dependency_count'] = train_df['hogar_nin'] + train_df['hogar_mayor']
train_df['dependency'] = train_df['dependency_count'] / train_df['adult']
train_df['child_percent'] = train_df['hogar_nin']/train_df['hogar_total']
train_df['elder_percent'] = train_df['hogar_mayor']/train_df['hogar_total']
train_df['adult_percent'] = train_df['hogar_adul']/train_df['hogar_total']
test_df['adult'] = test_df['hogar_adul'] - test_df['hogar_mayor']
test_df['dependency_count'] = test_df['hogar_nin'] + test_df['hogar_mayor']
test_df['dependency'] = test_df['dependency_count'] / test_df['adult']
test_df['child_percent'] = test_df['hogar_nin']/test_df['hogar_total']
test_df['elder_percent'] = test_df['hogar_mayor']/test_df['hogar_total']
test_df['adult_percent'] = test_df['hogar_adul']/test_df['hogar_total']

train_df['rent_per_adult'] = train_df['v2a1']/train_df['hogar_adul']
train_df['rent_per_person'] = train_df['v2a1']/train_df['hhsize']
test_df['rent_per_adult'] = test_df['v2a1']/test_df['hogar_adul']
test_df['rent_per_person'] = test_df['v2a1']/test_df['hhsize']

train_df['overcrowding_room_and_bedroom'] = (train_df['hacdor'] + train_df['hacapo'])/2
test_df['overcrowding_room_and_bedroom'] = (test_df['hacdor'] + test_df['hacapo'])/2

train_df['no_appliances'] = train_df['refrig'] + train_df['computer'] + train_df['television']
test_df['no_appliances'] = test_df['refrig'] + test_df['computer'] + test_df['television']

train_df['r4h1_percent_in_male'] = train_df['r4h1'] / train_df['r4h3']
train_df['r4m1_percent_in_female'] = train_df['r4m1'] / train_df['r4m3']
train_df['r4h1_percent_in_total'] = train_df['r4h1'] / train_df['hhsize']
train_df['r4m1_percent_in_total'] = train_df['r4m1'] / train_df['hhsize']
train_df['r4t1_percent_in_total'] = train_df['r4t1'] / train_df['hhsize']
test_df['r4h1_percent_in_male'] = test_df['r4h1'] / test_df['r4h3']
test_df['r4m1_percent_in_female'] = test_df['r4m1'] / test_df['r4m3']
test_df['r4h1_percent_in_total'] = test_df['r4h1'] / test_df['hhsize']
test_df['r4m1_percent_in_total'] = test_df['r4m1'] / test_df['hhsize']
test_df['r4t1_percent_in_total'] = test_df['r4t1'] / test_df['hhsize']

train_df['rent_per_room'] = train_df['v2a1']/train_df['rooms']
train_df['bedroom_per_room'] = train_df['bedrooms']/train_df['rooms']
train_df['elder_per_room'] = train_df['hogar_mayor']/train_df['rooms']
train_df['adults_per_room'] = train_df['adult']/train_df['rooms']
train_df['child_per_room'] = train_df['hogar_nin']/train_df['rooms']
train_df['male_per_room'] = train_df['r4h3']/train_df['rooms']
train_df['female_per_room'] = train_df['r4m3']/train_df['rooms']
train_df['room_per_person_household'] = train_df['hhsize']/train_df['rooms']

test_df['rent_per_room'] = test_df['v2a1']/test_df['rooms']
test_df['bedroom_per_room'] = test_df['bedrooms']/test_df['rooms']
test_df['elder_per_room'] = test_df['hogar_mayor']/test_df['rooms']
test_df['adults_per_room'] = test_df['adult']/test_df['rooms']
test_df['child_per_room'] = test_df['hogar_nin']/test_df['rooms']
test_df['male_per_room'] = test_df['r4h3']/test_df['rooms']
test_df['female_per_room'] = test_df['r4m3']/test_df['rooms']
test_df['room_per_person_household'] = test_df['hhsize']/test_df['rooms']

train_df['rent_per_bedroom'] = train_df['v2a1']/train_df['bedrooms']
train_df['edler_per_bedroom'] = train_df['hogar_mayor']/train_df['bedrooms']
train_df['adults_per_bedroom'] = train_df['adult']/train_df['bedrooms']
train_df['child_per_bedroom'] = train_df['hogar_nin']/train_df['bedrooms']
train_df['male_per_bedroom'] = train_df['r4h3']/train_df['bedrooms']
train_df['female_per_bedroom'] = train_df['r4m3']/train_df['bedrooms']
train_df['bedrooms_per_person_household'] = train_df['hhsize']/train_df['bedrooms']

test_df['rent_per_bedroom'] = test_df['v2a1']/test_df['bedrooms']
test_df['edler_per_bedroom'] = test_df['hogar_mayor']/test_df['bedrooms']
test_df['adults_per_bedroom'] = test_df['adult']/test_df['bedrooms']
test_df['child_per_bedroom'] = test_df['hogar_nin']/test_df['bedrooms']
test_df['male_per_bedroom'] = test_df['r4h3']/test_df['bedrooms']
test_df['female_per_bedroom'] = test_df['r4m3']/test_df['bedrooms']
test_df['bedrooms_per_person_household'] = test_df['hhsize']/test_df['bedrooms']

train_df['tablet_per_person_household'] = train_df['v18q1']/train_df['hhsize']
train_df['phone_per_person_household'] = train_df['qmobilephone']/train_df['hhsize']
test_df['tablet_per_person_household'] = test_df['v18q1']/test_df['hhsize']
test_df['phone_per_person_household'] = test_df['qmobilephone']/test_df['hhsize']

train_df['age_12_19'] = train_df['hogar_nin'] - train_df['r4t1']
test_df['age_12_19'] = test_df['hogar_nin'] - test_df['r4t1']    

train_df['escolari_age'] = train_df['escolari']/train_df['age']
test_df['escolari_age'] = test_df['escolari']/test_df['age']

train_df['rez_esc_escolari'] = train_df['rez_esc']/train_df['escolari']
train_df['rez_esc_r4t1'] = train_df['rez_esc']/train_df['r4t1']
train_df['rez_esc_r4t2'] = train_df['rez_esc']/train_df['r4t2']
train_df['rez_esc_r4t3'] = train_df['rez_esc']/train_df['r4t3']
train_df['rez_esc_age'] = train_df['rez_esc']/train_df['age']
test_df['rez_esc_escolari'] = test_df['rez_esc']/test_df['escolari']
test_df['rez_esc_r4t1'] = test_df['rez_esc']/test_df['r4t1']
test_df['rez_esc_r4t2'] = test_df['rez_esc']/test_df['r4t2']
test_df['rez_esc_r4t3'] = test_df['rez_esc']/test_df['r4t3']
test_df['rez_esc_age'] = test_df['rez_esc']/test_df['age']


FeatuersAggregation
====

In [ ]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()

aggr_mean_list = ['rez_esc', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 'parentesco2',
             'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12',
             'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',]

other_list = ['escolari', 'age', 'escolari_age']

for item in aggr_mean_list:
    group_train_mean = train_df[item].groupby(train_df['idhogar']).mean()
    group_test_mean = test_df[item].groupby(test_df['idhogar']).mean()
    new_col = item + '_aggr_mean'
    df_train[new_col] = group_train_mean
    df_test[new_col] = group_test_mean

for item in other_list:
    for function in ['mean','std','min','max','sum']:
        group_train = train_df[item].groupby(train_df['idhogar']).agg(function)
        group_test = test_df[item].groupby(test_df['idhogar']).agg(function)
        new_col = item + '_' + function
        df_train[new_col] = group_train
        df_test[new_col] = group_test

print (df_train.shape)
print (df_test.shape)

In [ ]:
df_test = df_test.reset_index()
df_train = df_train.reset_index()

train_agg = pd.merge(train_df, df_train, on='idhogar')
test = pd.merge(test_df, df_test, on='idhogar')

In [ ]:
print (train_agg.shape)
print (test.shape)

In [ ]:
train = train_agg.query('parentesco1==1')
train.head(10)

In [ ]:
useless_feats=['tamhog',
             'agesq', 
             'hogar_adul', 
             'SQBescolari', 
             'SQBage', 
             'SQBhogar_total', 
             'SQBedjefe', 
             'SQBhogar_nin', 
             'SQBovercrowding',
             'SQBdependency',
             'SQBmeaned'
            ]

In [ ]:
submission = test[['Id']]

In [ ]:
#Remove useless feature to reduce dimension
train.drop(columns=useless_feats, inplace=True)
test.drop(columns=useless_feats, inplace=True)

In [ ]:
correlation = train.corr()
correlation = correlation['Target'].sort_values(ascending=False)
print('The most 20 positive feature: \n')
print( correlation.head(20))
print('*'*50)
print('The most 30 negative feature: \n')
print (correlation.tail(10))

In [ ]:
train.head()

In [ ]:
feats = [f for f in train.columns if f not in ['Target','Id','idhogar','index']]

In [ ]:
train['dependency'] = train['dependency'].replace({np.inf: 0})
test['dependency'] = test['dependency'].replace({np.inf: 0})

In [ ]:
train[feats].head(20)

In [ ]:
test[feats].head(20)

In [ ]:
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

LightGBM
======

In [ ]:
y = train['Target']

clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.01, objective='multiclass',
                             random_state=None, silent=True, metric='None', 
                             n_jobs=4, n_estimators=5000, class_weight='balanced',
                             colsample_bytree =  0.93, min_child_samples = 95,
                             num_leaves = 14, subsample = 0.96)

kf = StratifiedKFold(n_splits=10, shuffle=True)

predicts_result = []
for train_index, valid_index in kf.split(train[feats], y):
    print("========================================")
    X_train, X_val = train[feats].iloc[train_index], train[feats].iloc[valid_index]
    y_train, y_val = y.iloc[train_index], y.iloc[valid_index]
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
            early_stopping_rounds=500, verbose=100)
    
    predicts_result.append(clf.predict(test[feats]))

For Stacking
===

In [ ]:
#train['Target'] = train['Target'].apply(lambda x: x-1)

In [ ]:
# LightGBM GBDT with KFold or Stratified KFold
#folds = KFold(n_splits= 5, shuffle=True, random_state=None)

#oof_preds = np.zeros((train.shape[0],4))
#sub_preds = np.zeros((test.shape[0],4))

#out_folder_train_prediction= pd.DataFrame()
#out_folder_test_prediction= pd.DataFrame()

#for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats], train['Target'])):
#    dtrain = lgb.Dataset(data=train[feats].iloc[train_idx], 
#                         label=train['Target'].iloc[train_idx],
#                         feature_name=feats)
#    dvalid = lgb.Dataset(data=train[feats].iloc[valid_idx],
#                         label=train['Target'].iloc[valid_idx],
#                         feature_name=feats)
#    
#    params = {
#            'objective': 'multiclass',
#            'boosting_type': 'gbdt',
#            'nthread': 4,
#            'learning_rate': 0.01,
#            'num_leaves': 14,
#            'colsample_bytree': 0.93,
#            'subsample': 0.96,
#            'class_weight': 'balanced',
#            'min_child_samples':95,
#            'num_class':4,
#            'is_balance': True,
#            #'silent':True,
#            'max_depth': -1,
#            'seed': 0,
#            'verbose': 100,
#            'metric': 'multi_logloss',
#        }
#    
#    watchlist = [dtrain, dvalid]
#    clf = lgb.train(
#            params=params,
#           train_set=dtrain,
#            num_boost_round=10000,
#            valid_sets=watchlist,
#            early_stopping_rounds=500,
#            verbose_eval=100
#        )
    
#    train_preds=pd.DataFrame()
#    train_preds['train_pred'] = clf.predict(train[feats].iloc[valid_idx])
#    train_preds['Id']=train['Id'].iloc[valid_idx].reset_index()['Id']
#    train_preds['Target']=train_df['Target'].iloc[valid_idx].reset_index().reset_index()['Target']
#    train_preds["folder"]=n_fold + 1
#    out_folder_train_prediction = pd.concat([out_folder_train_prediction, train_preds], axis=0)
#    test_preds=pd.DataFrame()
#    test_preds=test[['Id']].copy()
#    test_preds['test_pred']= clf.predict(test[feats]) 
#    test_preds["folder"]=n_fold + 1
#    out_folder_test_prediction = pd.concat([out_folder_test_prediction, test_preds], axis=0)
    
#    oof_preds[valid_idx] = clf.predict(train[feats].iloc[valid_idx])
#    sub_preds += clf.predict(test[feats]) / folds.n_splits

#    print('Fold %2d log_loss : %.6f' % (n_fold + 1, log_loss(train['Target'].iloc[valid_idx], oof_preds[valid_idx])))
#    del clf, dtrain, dvalid
#    gc.collect()

#print('Full log_loss score %.6f' % log_loss(train['Target'], oof_preds))
#out_folder_train_prediction.to_csv('out_folder_train_prediction_lgb.csv',index= False)
#out_folder_test_prediction.to_csv('out_folder_test_prediction_lgb.csv',index= False)

Feature Importance
----

In [ ]:
indices = np.argsort(clf.feature_importances_)[::-1]
indices = indices[:100]

# Visualise these with a barplot
plt.subplots(figsize=(20, 40))
g = sns.barplot(y=train.columns[indices], x = clf.feature_importances_[indices], orient='h')
g.set_xlabel("Relative importance",fontsize=15)
g.set_ylabel("Features",fontsize=12)
g.tick_params(labelsize=15)
g.set_title("LightGBM feature importance");

In [ ]:
submission['Target'] = np.array(predicts_result).mean(axis=0).round().astype(int)
submission.to_csv('submission_lightgbm.csv', index = False)

In [ ]:
def verify_submission(submission, sample_submission):
    assert submission.shape == sample_submission.shape, \
        'Expected submission to have shape {} but got {}'.format(sample_submission.shape, submission.shape)

    for submission_id, correct_id in zip(submission['Id'].values, sample_submission['Id'].values):
        assert correct_id == submission_id, \
            'Wrong id: expected {} but got {}'.format(correct_id, submission_id)
        
sample_submission=pd.read_csv('../input/sample_submission.csv')
verify_submission(submission, sample_submission)

In [ ]:
submission.Target.value_counts(normalize=True)

TOP 200 features
====


In [ ]:
train.columns[indices]

In [ ]:
indices = np.argsort(clf.feature_importances_)[::-1]
indices = indices[:200]
impo200_feats=[f for f in train.columns if f in train.columns[indices]]

In [ ]:
impo200_feats.remove('Target')

In [ ]:
train2=train[impo200_feats]
train2.drop(columns=['Id'], inplace=True)
train2.drop(columns=['idhogar'], inplace=True)
test2=test[impo200_feats]
test2.drop(columns=['Id'], inplace=True)
test2.drop(columns=['idhogar'], inplace=True)

In [ ]:
#y=train2['Target']
clf2 = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.01, objective='multiclass',
                             random_state=None, silent=True, metric='None', 
                             n_jobs=4, n_estimators=5000, class_weight='balanced',
                             colsample_bytree =  0.93, min_child_samples = 95,
                             num_leaves = 14, subsample = 0.96)

kf = StratifiedKFold(n_splits=10, shuffle=True)

predicts_result = []
for train_index, valid_index in kf.split(train2, y):
    print("========================================")
    X_train, X_val = train2.iloc[train_index], train2.iloc[valid_index]
    y_train, y_val = y.iloc[train_index], y.iloc[valid_index]
    clf2.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
            early_stopping_rounds=500, verbose=100)
    
    predicts_result.append(clf2.predict(test2))

In [ ]:
submission_top = test[['Id']]
submission_top['Target'] = np.array(predicts_result).mean(axis=0).round().astype(int)
submission_top.to_csv('submission_top200feats_lgbm.csv', index = False)

In [ ]:
submission_top.Target.value_counts(normalize=True)

Top 150
====

In [ ]:
indices = np.argsort(clf.feature_importances_)[::-1]
indices = indices[:150]
impo150_feats=[f for f in train.columns if f in train.columns[indices]]

In [ ]:
train3=train[impo150_feats]
train3.drop(columns=['Id'], inplace=True)
train3.drop(columns=['idhogar'], inplace=True)
test3=test[impo150_feats]
test3.drop(columns=['Id'], inplace=True)
test3.drop(columns=['idhogar'], inplace=True)

In [ ]:
clf3 = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.01, objective='multiclass',
                             random_state=None, silent=True, metric='None', 
                             n_jobs=4, n_estimators=5000, class_weight='balanced',
                             colsample_bytree =  0.93, min_child_samples = 95,
                             num_leaves = 14, subsample = 0.96)

kf = StratifiedKFold(n_splits=10, shuffle=True)

predicts_result = []
for train_index, valid_index in kf.split(train3, y):
    print("========================================")
    X_train, X_val = train3.iloc[train_index], train3.iloc[valid_index]
    y_train, y_val = y.iloc[train_index], y.iloc[valid_index]
    clf3.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
            early_stopping_rounds=500, verbose=100)
    
    predicts_result.append(clf3.predict(test3))

submission_top = test[['Id']]
submission_top['Target'] = np.array(predicts_result).mean(axis=0).round().astype(int)
submission_top.to_csv('submission_top150feats_lgbm.csv', index = False)

In [ ]:
submission_top.Target.value_counts(normalize=True)

KNN
====

In [ ]:
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import preprocessing

Imputation
===

In [ ]:
feats = [f for f in train.columns if f not in ['Target','Id','idhogar','index']]

imputer = sklearn.preprocessing.Imputer(strategy = "median")
imputer.fit(train[feats])
trainImputed = train[feats].copy()
trainImputed = pd.DataFrame(imputer.transform(train[feats]))
trainImputed.columns = train[feats].columns
trainImputed.index = train[feats].index

In [ ]:
testImputed = test[feats].copy()
testImputed = pd.DataFrame(imputer.transform(test[feats]))
testImputed.columns = test[feats].columns
testImputed.index = test[feats].index

In [ ]:
testImputed.head()

In [ ]:
trainImputed.head(15)

In [ ]:
np.where(np.isinf(trainImputed))

In [ ]:
where_are_inf = np.isinf(trainImputed)
trainImputed[where_are_inf] = 0

In [ ]:
#np.where(np.isnan(testImputed))
where_are_inf = np.isinf(testImputed)
testImputed[where_are_inf] = 0

In [ ]:
#Find k
Xtrain = trainImputed
Ytrain = train['Target']
list = []

i=10
while (i <= 600):
    knn = KNeighborsClassifier(n_neighbors=i) 

    knn.fit(Xtrain,Ytrain)
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=7)
    accuracy = sum(scores)/len(scores)
    list.append([i,accuracy])
    i += 10
    
pd.DataFrame(list, columns=["k","accuracy"]).plot(x="k",y="accuracy",style="")

In [ ]:
pd.DataFrame(list, columns=["k","accuracy"])["accuracy"].max()
#best k:100

In [ ]:
Ytest=""

knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(Xtrain,Ytrain)
Ytest = knn.predict(testImputed)

In [ ]:
submission_knn = test[['Id']]
submission_knn["Target"] = Ytest.tolist()

In [ ]:
submission_knn.Target.value_counts(normalize=True)
submission_knn.to_csv("submission_knn.csv", index=False)

XGBoost
====

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train[feats],  train['Target'], test_size=0.1, random_state=1)

In [ ]:
model = xgb.XGBClassifier( objective='multiclass', 
                          n_jobs=4,
                          silent=True,
                          n_estimators=10000,
                          class_weight='balanced', 
                          learning_rate=0.01, 
                          scale_pos_weight = 1).fit(train[feats],  train['Target'])
y_trainpred_xg = model.predict(X_train)
y_testpred_xg = model.predict(X_valid)

In [ ]:
def evaluate(y_train, y_valid, y_trainpred, y_testpred):
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import f1_score

    print("MAE :: ", mean_absolute_error(y_train, y_trainpred))
    print("Train Accuracy :: ", accuracy_score(y_train, y_trainpred)*100) 
    print("Test Accuracy  :: ", accuracy_score(y_valid, y_testpred) * 100)
    print("Confusion matrix :: \n", confusion_matrix(y_valid, y_testpred))

In [ ]:
evaluate(y_train, y_valid, y_trainpred_xg, y_testpred_xg)

In [ ]:
preds = model.predict(test[feats])

In [ ]:
submission_xgb = pd.DataFrame()
submission_xgb['Id'] = test['Id']
submission_xgb['Target'] = preds

In [ ]:
submission_xgb.head()

In [ ]:
submission_xgb.to_csv('submissions_xgb.csv', index=False)

In [ ]:
submission_xgb.Target.value_counts(normalize=True)

cross validation
===

In [ ]:
train['Target'] = train['Target'].apply(lambda x: x+1)

In [ ]:
y = train['Target']
kf = StratifiedKFold(n_splits=10, shuffle=True)

model = xgb.XGBClassifier( objective='multiclass', 
                          n_jobs=4,
                          max_depth=5,
                          silent=True,
                          n_estimators=10000,
                          subsample = 0.9,
                          colsample_bytree =  0.8,
                          min_child_weight = 1,
                          class_weight='balanced', 
                          learning_rate=0.01, 
                          scale_pos_weight = 1).fit(train[feats],  y)

predicts_result = []

for train_index, valid_index in kf.split(train[feats], y):
    print("========================================")
    X_train, X_val = train[feats].iloc[train_index], train[feats].iloc[valid_index]
    y_train, y_val = y.iloc[train_index], y.iloc[valid_index]
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
            early_stopping_rounds=500, verbose=100)
    
    predicts_result.append(model.predict(test[feats]))


In [ ]:
submission_xgb_cv = test[['Id']]
submission_xgb_cv['Target'] = np.array(predicts_result).mean(axis=0).round().astype(int)
submission_xgb_cv['Target'] = submission_xgb_cv['Target'].apply(lambda x: x-1)
submission_xgb_cv.to_csv('submission_xgb_cv.csv', index = False)

In [ ]:
submission_xgb_cv.head()

In [ ]:
submission_xgb_cv.Target.value_counts(normalize=True)

Ensembel
===

In [ ]:
from sklearn.linear_model import LogisticRegression
from scipy.special import expit, logit

In [ ]:
sub1=pd.read_csv('submission_lightgbm.csv') 
sub1.columns=['Id', 'Target1']

In [ ]:
sub2=pd.read_csv('submission_top200feats_lgbm.csv') 
sub2.columns=['Id', 'Target2']

In [ ]:
sub3=pd.read_csv('submission_top150feats_lgbm.csv') 
sub3.columns=['Id', 'Target3']

In [ ]:
dfs = [sub1,sub2, sub3]

In [ ]:
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='Id'),dfs)

In [ ]:
df_final

In [ ]:
df_final[[ 'Target1','Target2','Target3']].corr()

In [ ]:
new_col=df_final.median(axis=1)

In [ ]:
new_col2=[int(x) for x in new_col]
df_final['Target']=new_col2

In [ ]:
df_final.drop(columns=['Target1','Target2','Target3'], inplace=True)

In [ ]:
df_final.to_csv('submission_ensemble.csv', index = False)

In [ ]:
df_final